In [1]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
import numpy as np
from sklearn import tree


In [2]:
data = pd.read_csv('data/iris.csv')
data['class'] = pd.Categorical(data['class'])
classCodes = dict(enumerate(data['class'].cat.categories))



Splitting data into training / test allows me to test the accuracy of my model on unseend data.

this split is random however to mainatin class raitios i have used stratisfied sampling.

random state = 1 allows randomisation to remain deterministic, this ensures that test data dos not bleed into the training data

In [3]:
X = data.drop(['class'], axis=1)
y = data['class']

In [4]:
import pickle

filename = "models/iris-nn.pickle"
clf = pickle.load(open(filename, "rb"))


In [5]:
import dice_ml


# Dataset for training an ML model
d = dice_ml.Data(dataframe=data,
                continuous_features=[
                    'sepal_length', 'sepal_width', 'petal_length', 'petal_width'
                ],

                 outcome_name='class')

# Pre-trained ML model
m = dice_ml.Model(model=clf, backend="sklearn", model_type='classifier')
# DiCE explanation instance
exp = dice_ml.Dice(d, m, method="genetic",)


In [6]:
def flattenArray(arr):
    # Create an empty 2D list to hold the converted values
    result = []

    # Loop through each sub-array in the input array
    for sub_arr in arr:
        # Loop through each element in the sub-array
        for element in sub_arr:
            # Append the element to the result list
            result.append(element)

    return result


def lookupClassLabel(array,labels):
    for i in range(len(array)):
        array[i][-1] = labels[array[i][-1]]
    return array


# print(lookupClassLabel(cf["cfs_list"],classCodes))


In [7]:
import json
import csv

counterFactuals = []
percentage_to_sample = 1 / (len(classCodes) - 1)

f = open('counterfactuals/iris-1.csv', 'w')
first = True
writer = csv.writer(f)
for classCode in classCodes:
    query_instances = data[data["class"].cat.codes == classCode]
    for desiredClass in classCodes:

        if (desiredClass == classCode):
            continue
        print(
            f" {classCodes[classCode]} => {classCodes[desiredClass]}({percentage_to_sample})")

        result = exp.generate_counterfactuals(
            query_instances.drop(['class'], axis=1).sample(
                frac=percentage_to_sample, random_state=1),
            total_CFs=1,
            desired_class=int(desiredClass),
            proximity_weight=5,
            sparsity_weight=0.2,
            diversity_weight=0.5,
            categorical_penalty=0.1,
        )
        result = json.loads(result.to_json())
        cfList = result["cfs_list"]
        cfList = flattenArray(cfList)
        cfList = lookupClassLabel(cfList, classCodes)

        if (first):
            print(result['feature_names_including_target'])
            writer.writerow(result['feature_names_including_target'])
            first = False
        writer.writerows(cfList)






 Iris-setosa => Iris-versicolor(0.5)


100%|██████████| 25/25 [00:02<00:00,  8.34it/s]


['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
 Iris-setosa => Iris-virginica(0.5)


100%|██████████| 25/25 [00:03<00:00,  8.13it/s]


 Iris-versicolor => Iris-setosa(0.5)


...

100%|██████████| 25/25 [00:02<00:00,  8.85it/s]


 Iris-virginica => Iris-setosa(0.5)


100%|██████████| 25/25 [00:02<00:00,  9.28it/s]


 Iris-virginica => Iris-versicolor(0.5)


100%|██████████| 25/25 [00:01<00:00, 13.31it/s]


Validate Cfs against model, should get 100% accruracy as these are informed by model

In [8]:
# from sklearn.preprocessing import LabelBinarizer


# cfdata = pd.read_csv('counterfactuals/iris-1.csv')
# cfdata.head()
# X = cfdata.drop(['class'], axis=1)
# y = cfdata['class']


# y_score = clf.predict_proba(X)

# label_binarizer = LabelBinarizer().fit(y)
# y_onehot_test = label_binarizer.transform(y)
# y_onehot_test.shape  # (n_samples, n_classes)

# for class_of_interest in label_binarizer.classes_:
#     class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]

#     RocCurveDisplay.from_predictions(
#         y_onehot_test[:, class_id],
#         y_score[:, class_id],
#         name=f"{class_of_interest} vs the rest",
#         color="darkorange",
#     )
#     plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
#     plt.axis("square")
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title(f"{class_of_interest} vs the rest")
#     plt.legend()
#     plt.show()


In [9]:
# import random



# samples = 1000
# rows = {}

# headings = list(data.columns.values)



# for columnName in headings:
#     rows[columnName] = []


# for _ in range(samples):


#     for columnName in headings:
#         # print(columnName, data[columnName].dtype.name)
#         if (data[columnName].dtype.name == 'category'):
#             value = None
#         elif (data[columnName].dtype.name == 'float64'):
#             value = random.uniform(
#                 data[columnName].min(), data[columnName].max())
#         rows[columnName].append(value)

# randomData = pd.DataFrame(data=rows, columns=data.columns)

# X = randomData.drop(['class'], axis=1) 
# randomData['class'] = clf.predict(X)

# print(randomData.groupby(['class']).size())



# randomData.to_csv('random-pertuabtions/iris-100.csv',index=False)
